In [1]:
####Heterogeneous Infomation Network Embbedding with Variable order Tucker Decomposition####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import itertools
import scipy
from datetime import time, datetime, timedelta
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
import re
import MeCab
import neologdn
import sys

np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [3]:
####データの設定####
##シナリオの読み込みとクレンジング
#データの読み込み
company = pd.read_csv("D:/Statistics/data/scenario/company.csv")
scenario_df1 = pd.read_csv("D:/Statistics/data/scenario/level1_scenario.csv")
scenario_df2 = pd.read_csv("D:/Statistics/data/scenario/level2_scenario.csv")
factor_df = pd.read_csv("D:/Statistics/data/scenario/Factor.csv")

#読み込んだデータをクレンジング
scenario_df1.columns = (scenario_df1.columns).str.replace(" ", "")
scenario_df2.columns = (scenario_df2.columns).str.replace(" ", "")
factor_df.columns = (factor_df.columns).str.replace(" ", "")
company.columns = (company.columns).str.replace(" ", "")
columns1 = np.array(scenario_df1.columns)
columns2 = np.array(scenario_df2.columns)
columns3 = np.array(factor_df.columns)
columns4 = np.array(company.columns)
for j in range(columns1.shape[0]):
    scenario_df1[columns1[j]] = scenario_df1[columns1[j]].str.replace("\"| ", "")
for j in range(columns2.shape[0]):
    scenario_df2[columns2[j]] = scenario_df2[columns2[j]].str.replace("\"| ", "")
for j in range(columns3.shape[0]):
    factor_df[columns3[j]] = factor_df[columns3[j]].str.replace("\"| ", "")
scenario_df1["c.ticker"] = np.array(scenario_df1["c.ticker"], dtype="int")
scenario_df2["c.ticker"] = np.array(scenario_df2["c.ticker"], dtype="int")
scenario_df1["c.name"] = pd.merge(scenario_df1[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df2["c.name"] = pd.merge(scenario_df2[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df21 = scenario_df2[["f2.area", "f2.predicate", "f2.item", "f2.trend", "f2.element", "a.name", "a.vector", "c.name", "c.ticker"]]
scenario_df21 = scenario_df21.rename(columns={"f2.area": "f.area", "f2.predicate": "f.predicate", 
                                              "f2.item": "f.item", "f2.trend": "f.trend", "f2.element": "f.element"})
scenario_df1 = pd.concat((scenario_df1, scenario_df21), axis=0)
del scenario_df21

In [ ]:
#itemがないシナリオをelemenで埋める
index = np.where(scenario_df1["f.item"]=="")[0]
scenario_df1["f.item"].iloc[index] = scenario_df1["f.element"].iloc[index]
index = np.where(scenario_df2["f1.item"]=="")[0]
scenario_df2["f1.item"].iloc[index] = scenario_df2["f1.element"].iloc[index]
index = np.where(scenario_df2["f2.item"]=="")[0]
scenario_df2["f2.item"].iloc[index] = scenario_df2["f2.element"].iloc[index]
factor_df["f1.item"].iloc[index] = factor_df["f1.element"].iloc[index]
index = np.where(factor_df["f2.item"]=="")[0]
factor_df["f2.item"].iloc[index] = factor_df["f2.element"].iloc[index]
scenario_df1 = scenario_df1.iloc[np.where(scenario_df1["f.item"]!="")]
scenario_df1.index = np.arange(scenario_df1.shape[0])
scenario_df2 = scenario_df2.iloc[np.where((scenario_df2["f1.item"]!="") & (scenario_df2["f2.item"]!=""))[0]]
scenario_df2.index = np.arange(scenario_df2.shape[0])
factor_df = factor_df.iloc[np.where((factor_df["f1.item"]!="") & (factor_df["f2.item"]!=""))[0]]
factor_df.index = np.arange(factor_df.shape[0])

#itemが同じ要因を削除
scenario_df2 = scenario_df2.iloc[np.where(scenario_df2["f1.item"]!=scenario_df2["f2.item"])[0]]
factor_df = factor_df.iloc[np.where(factor_df["f1.item"]!=factor_df["f2.item"])[0]]
scenario_df2.index = np.arange(scenario_df2.shape[0])
factor_df.index = np.arange(factor_df.shape[0])

In [4]:
#itemがないシナリオを削除
scenario_df1 = scenario_df1.iloc[np.where(scenario_df1["f.item"]!="")]
scenario_df1.index = np.arange(scenario_df1.shape[0])
scenario_df2 = scenario_df2.iloc[np.where((scenario_df2["f1.item"]!="") & (scenario_df2["f2.item"]!=""))[0]]
scenario_df2.index = np.arange(scenario_df2.shape[0])
factor_df = factor_df.iloc[np.where((factor_df["f1.item"]!="") & (factor_df["f2.item"]!=""))[0]]
factor_df.index = np.arange(factor_df.shape[0])

#itemが同じ要因を削除
scenario_df2 = scenario_df2.iloc[np.where(scenario_df2["f1.item"]!=scenario_df2["f2.item"])[0]]
factor_df = factor_df.iloc[np.where(factor_df["f1.item"]!=factor_df["f2.item"])[0]]
scenario_df2.index = np.arange(scenario_df2.shape[0])
factor_df.index = np.arange(factor_df.shape[0])

In [5]:
##解析対象のレコードを抽出して、名寄を行う
#scenario1のレコードを抽出
index_dup = np.array(np.where(scenario_df1[["f.item", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario1 = scenario_df1[["f.item", "c.name", "c.ticker"]].iloc[index_dup]
scenario1.index = np.arange(scenario1.shape[0])

#scenario2のレコードを抽出
index_dup = np.array(np.where(scenario_df2[["f1.item", "f2.item", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario2 = scenario_df2[["f1.item", "f2.item", "c.name", "c.ticker"]].iloc[index_dup]
scenario2.index = np.arange(scenario2.shape[0])

#factorのレコードを抽出
index_dup = np.array(np.where(factor_df[["f1.item", "f2.item"]].duplicated()==False)[0], dtype="int")
factor = factor_df[["f1.item", "f2.item"]].iloc[index_dup]
factor.index = np.arange(factor.shape[0])

In [6]:
##要因および企業のidを生成
#itemのidを作成
temp = pd.concat((scenario1["f.item"], scenario2["f1.item"], 
                  scenario2["f2.item"], factor["f1.item"], factor["f2.item"]), axis=0)
temp_freq = temp.value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); item_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp), target))[0]]
item_entity = np.append(np.unique(temp)[1:], "")
item_id = np.arange(item_entity.shape[0])
item = item_entity.shape[0]-1
item_df = pd.DataFrame({"id": item_id, "item": item_entity})

#企業のidを作成
temp = pd.concat((scenario1[["c.name", "c.ticker"]], scenario2[["c.name", "c.ticker"]]), axis=0)
temp_freq = temp["c.ticker"].value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); company_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp["c.ticker"]), target))[0]]
temp.index = np.arange(temp.shape[0])
company_df = temp.iloc[np.where(temp.duplicated()==False)[0]]
company_df = company_df.sort_values("c.ticker")
company_df.index = np.arange(company_df.shape[0])
company_df["id"] = np.arange(company_df.shape[0])
company_df = company_df[["id", "c.ticker", "c.name"]]
company_df.columns = ["id", "ticker", "company"]
company_id = np.array(company_df["id"], dtype="int")

#頻度のデータフレームを作成
company_freq = pd.DataFrame({"ticker": np.array(company_freq.index, dtype="int"), "freq": np.array(company_freq, dtype="int")})
company_freq = pd.merge(company_freq, company_df, on="ticker", how="left")[["id", "ticker", "company", "freq"]]
item_freq = pd.DataFrame({"item": np.array(item_freq.index), "freq": np.array(item_freq, dtype="int")})
item_freq = pd.merge(item_freq, item_df, on="item", how="left")[["id", "item", "freq"]]

In [7]:
##要因-企業間のネットワークデータを生成
#要因-企業間のネットワークを生成
item_flag = pd.merge(scenario1[["f.item"]], item_df[:item_df.shape[0]-1], left_on="f.item", right_on="item", how="left")[["id", "item"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_fc1 = np.array(item_flag["id"].iloc[index_get], dtype="int16")
company_fc1 = np.array(company_flag["id"].iloc[index_get], dtype="int16")
n1 = index_get.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_fc1]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_company = company_df.iloc[company_fc1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fc = pd.concat((temp_item, temp_company), axis=1)
del temp_item, temp_company

In [8]:
#要因-要因間のネットワークを生成
item_flag1 = pd.merge(factor[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(factor[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False))[0], dtype="int")
item_ff11 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ff12 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
n2 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ff11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ff12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
ff = pd.concat((temp_item1, temp_item2), axis=1)
del temp_item1, temp_item2

In [9]:
#要因-要因-企業間のネットワークを生成
item_flag1 = pd.merge(scenario2[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(scenario2[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
company_flag = pd.merge(scenario2[["c.ticker"]], company_df,
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False) & 
                              (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_ffc11 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ffc12 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
company_ffc1 = np.array(company_flag["id"].iloc[index_get], dtype="int16")
n3 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ffc11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ffc12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_company = company_df.iloc[company_ffc1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
ffc = pd.concat((temp_item1, temp_item2, temp_company), axis=1)
del temp_item1, temp_item2, temp_company

In [10]:
##要因-企業-要因のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_company = np.unique(company_fc1)
n = get_company.shape[0]
company_list = [i for i in range(n)]
item_list1 = [i for i in range(n)]
item_list2 = [i for i in range(n)]

for i in range(n):
    #対象のitemとelementを抽出
    index = np.array(np.where(company_fc1==get_company[i])[0], dtype="int")
    get_item = item_fc1[index]
    if get_item.shape[0] <= 1:
        company_list[i] = np.array([], dtype="int16")
        item_list1[i] = np.array([], dtype="int16")
        item_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_item.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    company_list[i] = np.repeat(get_company[i], get_combine1.shape[0])
    item_list1[i] = get_item[get_combine1]
    item_list2[i] = get_item[get_combine2]
    
#リストを変換
company_fcf1 = np.array(list(itertools.chain(*[company_list[i] for i in range(n)])))
item_fcf11 = np.array(list(itertools.chain(*[item_list1[i] for i in range(n)])))
item_fcf12 = np.array(list(itertools.chain(*[item_list2[i] for i in range(n)])))
n4 = company_fcf1.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_fcf11]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_fcf12]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_company = company_df.iloc[company_fcf1]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fcf = pd.concat((temp_company, temp_item1, temp_item2), axis=1)
del item_list1, item_list2, company_list
del temp_item1, temp_item2, temp_company

In [11]:
##企業-要因-企業のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_item = np.unique(item_fc1); get_item = get_item[:get_item.shape[0]-1]
n = get_item.shape[0]
item_list = [i for i in range(n)]
company_list1 = [i for i in range(n)]
company_list2 = [i for i in range(n)]

for i in range(n):
    #対象の企業を抽出
    index = np.array(np.where(item_fc1==get_item[i])[0], dtype="int")
    get_company = company_fc1[index]
    if get_company.shape[0] <= 1:
        item_list[i] = np.array([], dtype="int16")
        company_list1[i] = np.array([], dtype="int16")
        company_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_company.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    item_list[i] = np.repeat(get_item[i], get_combine1.shape[0])
    company_list1[i] = get_company[get_combine1]
    company_list2[i] = get_company[get_combine2]
    
#リストを変換
item_cfc1 = np.array(list(itertools.chain(*[item_list[i] for i in range(n)])))
company_cfc11 = np.array(list(itertools.chain(*[company_list1[i] for i in range(n)])))
company_cfc12 = np.array(list(itertools.chain(*[company_list2[i] for i in range(n)])))
n5 = item_cfc1.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_cfc1]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_company1 = company_df.iloc[company_cfc11]
temp_company1.columns = ["company_id1", "ticker1", "company1"]; temp_company1.index = np.arange(temp_company1.shape[0])
temp_company2 = company_df.iloc[company_cfc12]
temp_company2.columns = ["company_id2", "ticker2", "company2"]; temp_company2.index = np.arange(temp_company2.shape[0])
cfc = pd.concat((temp_company1, temp_company2, temp_item), axis=1)
del item_list, company_list1, company_list2
del temp_item, temp_company1, temp_company2

In [12]:
#データフレームを出力
fc.to_excel("D:/Statistics/data/scenario_link/factor-company2.xlsx")
ff.to_excel("D:/Statistics/data/scenario_link/factor-factor2.xlsx")
ffc.to_excel("D:/Statistics/data/scenario_link/factor-factor-company2.xlsx")
fcf.to_excel("D:/Statistics/data/scenario_link/factor-company-factor2.xlsx")
cfc.to_excel("D:/Statistics/data/scenario_link/company-factor-company2.xlsx")

In [13]:
##負例サンプリングにより新しいサンプルを生成
#サンプリング確率を設定
s = 3
item_prob = np.power(item_freq["freq"], 0.75) / np.sum(np.power(item_freq["freq"], 0.75))
company_prob = np.power(company_freq["freq"], 0.75) / np.sum(np.power(company_freq["freq"], 0.75))

#要因-企業間の負例をサンプリング
item_fc0 = np.array(np.dot(np.random.multinomial(1, item_prob, n1*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fc0 = np.array(item_freq["id"].iloc[item_fc0], dtype="int16")
company_fc0 = np.array(np.dot(np.random.multinomial(1, company_prob, n1*s), np.arange(company_prob.shape[0])), dtype="int16")
company_fc0 = np.array(company_freq["id"].iloc[company_fc0], dtype="int16")
temp_df1 = pd.concat((fc[["item_id", "company_id"]], pd.DataFrame({"flag": np.repeat(1, n1)})), axis=1)
temp_df2 = pd.DataFrame({"item_id": item_fc0, "company_id": company_fc0, "flag": np.repeat(0, n1*s)})
temp_df = pd.concat((temp_df1, temp_df2), axis=0)
index = np.where((temp_df[["item_id", "company_id"]].duplicated()==False) & (temp_df["flag"]==0))[0]
item_fc0 = np.array(temp_df["item_id"].iloc[index], dtype="int16")
company_fc0 = np.array(temp_df["company_id"].iloc[index], dtype="int16")

#要因-要因間の負例をサンプリング
item_ff01 = np.array(np.dot(np.random.multinomial(1, item_prob, n2*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ff01 = np.array(item_freq["id"].iloc[item_ff01], dtype="int16")
item_ff02 = np.array(np.dot(np.random.multinomial(1, item_prob, n2*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ff02 = np.array(item_freq["id"].iloc[item_ff02], dtype="int16")
temp_df1 = pd.concat((ff[["item_id1", "item_id2"]], pd.DataFrame({"flag": np.repeat(1, n2)})), axis=1)
temp_df2 = pd.DataFrame({"item_id1": item_ff01, "item_id2": item_ff02, "flag": np.repeat(0, n2*s)})
temp_df = pd.concat((temp_df1, temp_df2), axis=0)
index = np.where((temp_df[["item_id1", "item_id2"]].duplicated()==False) & (temp_df["flag"]==0))[0]
item_ff01 = np.array(temp_df["item_id1"].iloc[index], dtype="int16")
item_ff02 = np.array(temp_df["item_id2"].iloc[index], dtype="int16")

#要因-要因-企業間の負例をサンプリング
item_ffc01 = np.array(np.dot(np.random.multinomial(1, item_prob, n3*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ffc01 = np.array(item_freq["id"].iloc[item_ffc01], dtype="int16")
item_ffc02 = np.array(np.dot(np.random.multinomial(1, item_prob, n3*s), np.arange(item_prob.shape[0])), dtype="int16")
item_ffc02 = np.array(item_freq["id"].iloc[item_ffc02], dtype="int16")
company_ffc0 = np.array(np.dot(np.random.multinomial(1, company_prob, n3*s), np.arange(company_prob.shape[0])), dtype="int16")
company_ffc0 = np.array(company_freq["id"].iloc[company_ffc0], dtype="int16")
temp_df1 = pd.concat((ffc[["item_id1", "item_id2", "company_id"]], pd.DataFrame({"flag": np.repeat(1, n3)})), axis=1)
temp_df2 = pd.DataFrame({"item_id1": item_ffc01, "item_id2": item_ffc02, "company_id": company_ffc0, "flag": np.repeat(0, n3*s)})
temp_df = pd.concat((temp_df1, temp_df2), axis=0)
index = np.where((temp_df[["item_id1", "item_id2", "company_id"]].duplicated()==False) & (temp_df["flag"]==0))[0]
item_ffc01 = np.array(temp_df["item_id1"].iloc[index], dtype="int16")
item_ffc02 = np.array(temp_df["item_id2"].iloc[index], dtype="int16")
company_ffc0 = np.array(temp_df["company_id"].iloc[index], dtype="int16")

#要因-企業-要因間の負例をサンプリング
item_fcf01 = np.array(np.dot(np.random.multinomial(1, item_prob, n4*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fcf01 = np.array(item_freq["id"].iloc[item_fcf01], dtype="int16")
item_fcf02 = np.array(np.dot(np.random.multinomial(1, item_prob, n4*s), np.arange(item_prob.shape[0])), dtype="int16")
item_fcf02 = np.array(item_freq["id"].iloc[item_fcf02], dtype="int16")
company_fcf0 = np.array(np.dot(np.random.multinomial(1, company_prob, n4*s), np.arange(company_prob.shape[0])), dtype="int16")
company_fcf0 = np.array(company_freq["id"].iloc[company_fcf0], dtype="int16")
temp_df1 = pd.concat((fcf[["item_id1", "item_id2", "company_id"]], pd.DataFrame({"flag": np.repeat(1, n4)})), axis=1)
temp_df2 = pd.DataFrame({"item_id1": item_fcf01, "item_id2": item_fcf02, "company_id": company_fcf0, "flag": np.repeat(0, n4*s)})
temp_df = pd.concat((temp_df1, temp_df2), axis=0)
index = np.where((temp_df[["item_id1", "item_id2", "company_id"]].duplicated()==False) & (temp_df["flag"]==0))[0]
item_fcf01 = np.array(temp_df["item_id1"].iloc[index], dtype="int16")
item_fcf02 = np.array(temp_df["item_id2"].iloc[index], dtype="int16")
company_fcf0 = np.array(temp_df["company_id"].iloc[index], dtype="int16")

#企業-要因-企業間の負例をサンプリング
item_cfc0 = np.array(np.dot(np.random.multinomial(1, item_prob, n5*s), np.arange(item_prob.shape[0])), dtype="int16")
item_cfc0 = np.array(item_freq["id"].iloc[item_cfc0], dtype="int16")
company_cfc01 = np.array(np.dot(np.random.multinomial(1, company_prob, n5*s), np.arange(company_prob.shape[0])), dtype="int16")
company_cfc01 = np.array(company_freq["id"].iloc[company_cfc01], dtype="int16")
company_cfc02 = np.array(np.dot(np.random.multinomial(1, company_prob, n5*s), np.arange(company_prob.shape[0])), dtype="int16")
company_cfc02 = np.array(company_freq["id"].iloc[company_cfc02], dtype="int16")
temp_df1 = pd.concat((cfc[["company_id1", "company_id2", "item_id"]], pd.DataFrame({"flag": np.repeat(1, n4)})), axis=1)
temp_df2 = pd.DataFrame({"company_id1": company_cfc01, "company_id2": company_cfc02, "item_id": item_cfc0, "flag": np.repeat(0, n5*s)})
temp_df = pd.concat((temp_df1, temp_df2), axis=0)
index = np.where((temp_df[["company_id1", "company_id2", "item_id"]].duplicated()==False) & (temp_df["flag"]==0))[0]
company_cfc01 = np.array(temp_df["company_id1"].iloc[index], dtype="int16")
company_cfc02 = np.array(temp_df["company_id2"].iloc[index], dtype="int16")
item_cfc0 = np.array(temp_df["item_id"].iloc[index], dtype="int16")
del temp_df1, temp_df2, temp_df, index

In [14]:
##応答変数を設定
#要因-企業間の応答変数を定義
item_fc = np.append(item_fc1, item_fc0)
company_fc = np.append(company_fc1, company_fc0)
y1 = np.append(np.repeat(1, n1), np.repeat(0, item_fc0.shape[0]))
N1 = y1.shape[0]

#要因-要因間の応答変数を定義
item_ff1 = np.append(item_ff11, item_ff01)
item_ff2 = np.append(item_ff12, item_ff02)
y2 = np.append(np.repeat(1, n2), np.repeat(0, item_ff01.shape[0]))
N2 = y2.shape[0]

#要因ー要因-企業間の応答変数を定義
item_ffc1 = np.append(item_ffc11, item_ffc01)
item_ffc2 = np.append(item_ffc12, item_ffc02)
company_ffc = np.append(company_ffc1, company_ffc0)
y3 = np.append(np.repeat(1, n3), np.repeat(0, item_ffc01.shape[0]))
N3 = y3.shape[0]

#要因ー企業-要因間の応答変数を定義
item_fcf1 = np.append(item_fcf11, item_fcf01)
item_fcf2 = np.append(item_fcf12, item_fcf02)
company_fcf = np.append(company_fcf1, company_fcf0)
y4 = np.append(np.repeat(1, n4), np.repeat(0, item_fcf01.shape[0]))
N4 = y4.shape[0]

#企業-要因-企業間の応答変数を定義
item_cfc = np.append(item_cfc1, item_cfc0)
company_cfc1 = np.append(company_cfc11, company_cfc01)
company_cfc2 = np.append(company_cfc12, company_cfc02)
y5 = np.append(np.repeat(1, n5), np.repeat(0, item_cfc0.shape[0]))
N5 = y5.shape[0]

In [15]:
##学習データとテストデータに分割
#要因-企業間のデータを分割
split = 4/5
index_learn = np.random.choice(N1, int(split * N1), replace=False)
index_test = np.delete(np.arange(N1), index_learn)
item_fc1 = item_fc[index_learn]; item_fc2 = item_fc[index_test]
company_fc1 = company_fc[index_learn]; company_fc2 = company_fc[index_test]
y11 = y1[index_learn]; y12 = y1[index_test]
N11 = index_learn.shape[0]; N12 = index_test.shape[0]

#要因-要因間のデータを分割
index_learn = np.random.choice(N2, int(split * N2), replace=False)
index_test = np.delete(np.arange(N2), index_learn)
item_ff11 = item_ff1[index_learn]; item_ff12 = item_ff1[index_test]
item_ff21 = item_ff2[index_learn]; item_ff22 = item_ff2[index_test]
y21 = y2[index_learn]; y22 = y2[index_test]
N21 = index_learn.shape[0]; N22 = index_test.shape[0]

#要因-要因-企業間のデータを分割
index_learn = np.random.choice(N3, int(split * N3), replace=False)
index_test = np.delete(np.arange(N3), index_learn)
item_ffc11 = item_ffc1[index_learn]; item_ffc12 = item_ffc1[index_test]
item_ffc21 = item_ffc2[index_learn]; item_ffc22 = item_ffc2[index_test]
company_ffc1 = company_ffc[index_learn]; company_ffc2 = company_ffc[index_test]
y31 = y3[index_learn]; y32 = y3[index_test]
N31 = index_learn.shape[0]; N32 = index_test.shape[0]

#要因-企業-要因間のデータを分割
index_learn = np.random.choice(N4, int(split * N4), replace=False)
index_test = np.delete(np.arange(N4), index_learn)
item_fcf11 = item_fcf1[index_learn]; item_fcf12 = item_fcf1[index_test]
item_fcf21 = item_fcf2[index_learn]; item_fcf22 = item_fcf2[index_test]
company_fcf1 = company_fcf[index_learn]; company_fcf2 = company_fcf[index_test]
y41 = y4[index_learn]; y42 = y4[index_test]
N41 = index_learn.shape[0]; N42 = index_test.shape[0]

#企業-要因-企業間のデータを分割
index_learn = np.random.choice(N5, int(split * N5), replace=False)
index_test = np.delete(np.arange(N5), index_learn)
item_cfc1 = item_cfc[index_learn]; item_cfc2 = item_cfc[index_test]
company_cfc11 = company_cfc1[index_learn]; company_cfc12 = company_cfc1[index_test]
company_cfc21 = company_cfc2[index_learn]; company_cfc22 = company_cfc2[index_test]
y51 = y5[index_learn]; y52 = y5[index_test]
N51 = index_learn.shape[0]; N52 = index_test.shape[0]
N = [N11, N21, N31, N41, N51]

In [16]:
##インデックスを設定
#要因と企業のユニーク数
item_n = item_df.shape[0]-1
company_n = company_df.shape[0]

#itemのインデックスを設定
item_list_fc = [i for i in range(item_n)]
item_list_ff1 = [i for i in range(item_n)]
item_list_ff2 = [i for i in range(item_n)]
item_list_ffc1 = [i for i in range(item_n)]
item_list_ffc2 = [i for i in range(item_n)]
item_list_fcf1 = [i for i in range(item_n)]
item_list_fcf2 = [i for i in range(item_n)]
item_list_cfc = [i for i in range(item_n)]
for i in range(item_n):
    item_list_fc[i] = np.array(np.where(item_fc1==i)[0], dtype="int")
    item_list_ff1[i] = np.array(np.where(item_ff11==i)[0], dtype="int")
    item_list_ff2[i] = np.array(np.where(item_ff21==i)[0], dtype="int")
    item_list_ffc1[i] = np.array(np.where(item_ffc11==i)[0], dtype="int")
    item_list_ffc2[i] = np.array(np.where(item_ffc21==i)[0], dtype="int")
    item_list_fcf1[i] = np.array(np.where(item_fcf11==i)[0], dtype="int")
    item_list_fcf2[i] = np.array(np.where(item_fcf21==i)[0], dtype="int")
    item_list_cfc[i] = np.array(np.where(item_cfc1==i)[0], dtype="int")
    
#企業のインデックスを設定
company_list_fc = [i for i in range(company_n)]
company_list_ffc = [i for i in range(company_n)]
company_list_fcf = [i for i in range(company_n)]
company_list_cfc1 = [i for i in range(company_n)]
company_list_cfc2 = [i for i in range(company_n)]
for i in range(company_n):
    company_list_fc[i] = np.array(np.where(company_fc1==i)[0], dtype="int")
    company_list_ffc[i] = np.array(np.where(company_ffc1==i)[0], dtype="int")
    company_list_fcf[i] = np.array(np.where(company_fcf1==i)[0], dtype="int")
    company_list_cfc1[i] = np.array(np.where(company_cfc11==i)[0], dtype="int")
    company_list_cfc2[i] = np.array(np.where(company_cfc21==i)[0], dtype="int")

In [26]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [27]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [28]:
##アルゴリズムの設定
type_n = 5
k = 10
k_vec = np.repeat(1, k)
R = 1000
keep = 2
burnin = int(200/keep)
iter = 0
disp = 10

In [29]:
##事前分布の設定
#階層モデルの事前分布
delta = np.repeat(0, k)
V = 0.1 * np.diag(np.ones(k))
nu = 1
s01 = 0.1
s01 = 0.1

#モデルパラメータの事前分布
alpha = np.repeat(0.0, k)
tau = 100 
s02 = 1.0
v02 = 1.0

In [30]:
##初期値の設定
#階層モデルの初期値
Cov_v = np.array(np.diag(np.repeat(0.2, k)), dtype="float32"); inv_Cov_v = np.linalg.inv(Cov_v)
Cov_i = np.array(np.diag(np.repeat(0.2, k)), dtype="float32"); inv_Cov_i = np.linalg.inv(Cov_i)
Cov_m = np.array(np.diag(np.repeat(0.2, k*k)), dtype="float32"); inv_Cov_m = np.linalg.inv(Cov_m)

#モデルパラメータの初期値
Sigma = np.array([1.0], dtype="float32")
beta = np.array(np.repeat(0.0, type_n), dtype="float32")
theta_v = np.random.multivariate_normal(alpha, np.diag(np.repeat(0.1, k)), company_n)
theta_i = np.random.multivariate_normal(alpha, np.diag(np.repeat(0.1, k)), item_n)
omega_fc = np.array(np.random.normal(0, 0.1, k*k).reshape(k, k), dtype="float32")
omega_ff = np.array(np.random.normal(0, 0.1, k*k).reshape(k, k), dtype="float32")
omega_ffc = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")
omega_fcf = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")
omega_cfc = np.array(np.random.normal(0, 0.1, k*k*k).reshape(k, k, k), dtype="float32")

In [31]:
##データの設定
#切断領域を定義
rho = 10.0
y = [y11, y21, y31, y41, y51]
a = [j for j in range(type_n)]
b = [j for j in range(type_n)]
for j in range(type_n):
    a[j] = np.array((1-y[j] )*(-rho) + y[j]*0, dtype="float32")
    b[j] = np.array(y[j]*rho + (1-y[j])*0, dtype="float32")
    
#特徴行列の割当インデックス
allocation1 = np.repeat(np.arange(k), k)
allocation2 = np.tile(np.arange(k), k)
allocation_omega = np.array([np.delete(np.arange(k), j).tolist() for j in range(k)])

In [32]:
##パラメータの格納用配列
BETA = np.zeros((int(R/keep), type_n), dtype="float32")
THETA_V = np.zeros((company_n, k, int(R/keep)), dtype="float32")
THETA_I = np.zeros((item_n, k, int(R/keep)), dtype="float32")
OMEGA_FC = np.zeros((k, k, int(R/keep)), dtype="float32")
OMEGA_FF = np.zeros((k, k, int(R/keep)), dtype="float32")
OMEGA_FFC = np.zeros((k, k, k, int(R/keep)), dtype="float32")
OMEGA_FCF = np.zeros((k, k, k, int(R/keep)), dtype="float32")
OMEGA_CFC = np.zeros((k, k, k, int(R/keep)), dtype="float32")

In [33]:
##対数尤度の基準値
#学習データに対する1パラメータモデルの対数尤度
y_learn = [y11, y21, y31, y41, y51]
LLst1 = np.repeat(0.0, type_n)
for j in range(type_n):
    Prob = np.mean(y_learn[j])
    LLst1[j] = np.sum(y_learn[j]*np.log(Prob)) + np.sum((1-y_learn[j])*np.log(1-Prob))
print(np.round(np.append(LLst1, np.sum(LLst1)), 1))

#テストデータに対する1パラメータモデルの対数尤度
y_test = [y12, y22, y32, y42, y52]
LLst0 = np.repeat(0.0, type_n)
for j in range(type_n):
    Prob = np.mean(y_test[j])
    LLst0[j] = np.sum(y_test[j]*np.log(Prob)) + np.sum((1-y_test[j])*np.log(1-Prob))
print(np.round(np.append(LLst0, np.sum(LLst0)), 1))

[ -13704.6  -15200.8  -56174.2  -73775.  -113032.2 -271886.9]
[ -3410.2  -3807.4 -13923.5 -18574.8 -28138.2 -67854.1]


In [34]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##切断正規分布から潜在効用を生成
    #特徴ベクトルをidで対応付ける
    theta_item1 = theta_i[item_fc1, ]; theta_company1 = theta_v[company_fc1, ]
    theta_item21 = theta_i[item_ff11, ]; theta_item22 = theta_i[item_ff21, ]
    theta_item31 = theta_i[item_ffc11, ]; theta_item32 = theta_i[item_ffc21, ]; theta_company3 = theta_v[company_ffc1, ]
    theta_item41 = theta_i[item_fcf11, ]; theta_item42 = theta_i[item_fcf21, ]; theta_company4 = theta_v[company_fcf1, ]
    theta_item5 = theta_i[item_cfc1, ]; theta_company51 = theta_v[company_cfc11, ]; theta_company52 = theta_v[company_cfc21, ]

    #タッカー分解の期待値
    uv1 = np.dot(np.dot(theta_i, omega_fc)[item_fc1, ] * theta_company1, k_vec)
    uv2 = np.dot(np.dot(theta_i, omega_ff)[item_ff11] * (theta_item22), k_vec)
    uv3 = np.array(np.repeat(0.0, N31), dtype="float32")
    uv4 = np.array(np.repeat(0.0, N41), dtype="float32")
    uv5 = np.array(np.repeat(0.0, N51), dtype="float32")
    for j in range(k):
        uv3 += np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ] * theta_item32 * theta_company3[:, j][:, np.newaxis], k_vec)
        uv4 += np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ] * theta_item42 * theta_company4[:, j][:, np.newaxis], k_vec)
        uv5 += np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
    uv = [uv1, uv2, uv3, uv4, uv5]

    #潜在効用を生成
    mu = [j for j in range(type_n)]
    U = [j for j in range(type_n)]
    for j in range(type_n):
        mu[j] = beta[j] + uv[j]
        U[j] = np.array(rtnorm(mu[j], Sigma, a[j], b[j], N[j]), dtype="float32")


    ##モデルの期待値をサンプリング
    for j in range(type_n):
        #モデル誤差を定義
        er_y = U[j] - uv[j]

        #正規分布から期待値をサンプリング
        weights = np.power(tau, 2) / (Sigma/N[j] + np.power(tau, 2))
        mu_par = weights * np.mean(er_y)
        beta[j] = np.random.normal(mu_par, weights*Sigma/N[j], 1)

    ##アイテムの特徴ベクトルをサンプリング
    #モデル誤差を定義
    er_y = [U[j] - beta[j] for j in range(type_n)]

    #推定対象のパラメータを周辺化する
    AR1 = np.dot(theta_v, omega_fc.T)[company_fc1, ]
    RA2 = np.dot(theta_i, omega_ff.T)[item_ff21, ]; AR2 = np.dot(theta_i, omega_ff)[item_ff11, ]
    ARB1 = np.array(np.zeros((N31, k)), dtype="float32"); BRA1 = np.array(np.zeros((N31, k)), dtype="float32")
    ARB2 = np.array(np.zeros((N41, k)), dtype="float32"); BRA2 = np.array(np.zeros((N41, k)), dtype="float32")
    ARB3 = np.array(np.zeros((N51, k)), dtype="float32")
    for j in range(k):
        ARB1 += np.dot(theta_i, omega_ffc[:, :, j].T)[item_ffc21, ] * theta_company3[:, j][:, np.newaxis]
        BRA1 += np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ] * theta_company3[:, j][:, np.newaxis]
        ARB2 += np.dot(theta_i, omega_fcf[:, :, j].T)[item_fcf21, ] * theta_company4[:, j][:, np.newaxis]
        BRA2 += np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ] * theta_company4[:, j][:, np.newaxis]
        ARB3 += np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * theta_company52[:, j][:, np.newaxis]

    #事後分布をサンプリング
    for i in range(item_n):
        #応答変数を定義
        y1 = er_y[0][item_list_fc[i]]; 
        y2 = np.append(er_y[1][item_list_ff1[i], ], er_y[1][item_list_ff2[i]])
        y3 = np.append(er_y[2][item_list_ffc1[i]], er_y[2][item_list_ffc2[i]])
        y4 = np.append(er_y[3][item_list_fcf1[i]], er_y[3][item_list_fcf2[i]])
        y5 = er_y[4][item_list_cfc[i]]
        y = np.hstack((y1, y2, y3, y4, y5))

        #入力変数を定義
        x1 = AR1[item_list_fc[i]]
        x2 = np.vstack((RA2[item_list_ff1[i], ], AR2[item_list_ff2[i], ]))
        x3 = np.vstack((ARB1[item_list_ffc1[i], ], BRA1[item_list_ffc2[i], ]))
        x4 = np.vstack((ARB2[item_list_fcf1[i], ], BRA2[item_list_fcf2[i], ]))
        x5 = ARB3[item_list_cfc[i], ]
        x = np.vstack((x1, x2, x3, x4, x5))

        #事後分布の期待値
        xy = np.dot(x.T, y)
        xxv = np.dot(x.T, x) + inv_Cov_i
        inv_xxv = np.array(np.linalg.inv(xxv), dtype="float64")
        theta_mu = np.dot(inv_xxv, xy + np.dot(inv_Cov_i, alpha))

        #多変量正規分布からパラメータをサンプリング
        theta_i[i, ] = np.random.multivariate_normal(theta_mu, Sigma*inv_xxv, 1).reshape(-1)
    theta_i = np.array(theta_i, dtype="float32")

    #特徴ベクトルをidで対応付ける
    theta_item1 = theta_i[item_fc1, ]
    theta_item31 = theta_i[item_ffc11, ]; theta_item32 = theta_i[item_ffc21, ]
    theta_item41 = theta_i[item_fcf11, ]; theta_item42 = theta_i[item_fcf21, ]
    theta_item5 = theta_i[item_cfc1, ]


    ##企業の特徴ベクトルをサンプリング
    #推定対象のパラメータを周辺化する
    AR1 = np.dot(theta_i, omega_fc)[item_fc1, ]
    ARB1 = np.array(np.zeros((N31, k)), dtype="float32")
    ARB2 = np.array(np.zeros((N41, k)), dtype="float32")
    ARB3 = np.array(np.zeros((N51, k)), dtype="float32"); BRA3 = np.array(np.zeros((N51, k)), dtype="float32")
    for j in range(k):
        ARB1[:, j] = np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ] * theta_item32, k_vec)
        ARB2[:, j] = np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ] * theta_item42, k_vec)
        ARB3 += np.dot(theta_i, omega_cfc[:, :, j].T)[item_cfc1, ] * theta_company52[:, j][:, np.newaxis]
        BRA3[:, j] = np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * theta_item5, k_vec)

    #事後分布をサンプリング
    for i in range(company_n):
        #応答変数を定義
        y1 = er_y[0][company_list_fc[i]]; 
        y3 = er_y[2][company_list_ffc[i]]
        y4 = er_y[3][company_list_fcf[i]]
        y5 = np.append(er_y[4][company_list_cfc1[i]], er_y[4][company_list_cfc2[i]])
        y = np.hstack((y1, y3, y4, y5))

        #入力変数を定義
        x1 = AR1[company_list_fc[i], ]
        x3 = ARB1[company_list_ffc[i], ]
        x4 = ARB2[company_list_fcf[i], ]
        x5 = np.vstack((ARB3[company_list_cfc1[i], ], BRA3[company_list_cfc2[i], ]))
        x = np.vstack((x1, x3, x4, x5))

        #事後分布の期待値
        xy = np.dot(x.T, y)
        xxv = np.dot(x.T, x) + inv_Cov_v
        inv_xxv = np.array(np.linalg.inv(xxv), dtype="float64")
        theta_mu = np.dot(inv_xxv, xy + np.dot(inv_Cov_v, alpha))

        #多変量正規分布からパラメータをサンプリング
        theta_v[i, ] = np.random.multivariate_normal(theta_mu, Sigma*inv_xxv, 1).reshape(-1)
    theta_v = np.array(theta_v, dtype="float32")

    #特徴ベクトルをidで対応付ける
    theta_company1 = theta_v[company_fc1, ]
    theta_company3 = theta_v[company_ffc1, ]
    theta_company4 = theta_v[company_fcf1, ]
    theta_company51 = theta_v[company_cfc11, ]; theta_company52 = theta_v[company_cfc21, ]


    ##双線形回帰行列をサンプリング
    #応答変数と入力変数を定義
    y1 = er_y[0]; y2 = er_y[1] 
    x1 = theta_item1[:, allocation1] * theta_company1[:, allocation2]
    x2 = theta_item21[:, allocation1] * theta_item22[:, allocation2]

    #事後分布の期待値
    xy1 = np.dot(x1.T, y1); xy2 = np.dot(x2.T, y2)
    xxv1 = np.dot(x1.T, x1) + inv_Cov_m; xxv2 = np.dot(x2.T, x2) + inv_Cov_m
    inv_xxv1 = np.linalg.inv(xxv1); inv_xxv2 = np.linalg.inv(xxv2)
    theta_mu1 = np.dot(inv_xxv1, xy1); theta_mu2 = np.dot(inv_xxv2, xy2)

    #多変量正規分布からパラメータをサンプリング
    omega_fc = np.random.multivariate_normal(theta_mu1, Sigma*inv_xxv1, 1).reshape(k, k)
    omega_ff = np.random.multivariate_normal(theta_mu2, Sigma*inv_xxv2, 1).reshape(k, k)


    ##コアテンソルをサンプリング
    #タッカー分解の期待値を展開
    uv_deploy1 = np.zeros((N31, k), dtype="float32")
    uv_deploy2 = np.zeros((N41, k), dtype="float32")
    uv_deploy3 = np.zeros((N51, k), dtype="float32")
    for j in range(k):
        uv_deploy1[:, j] = np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ]
                                  * theta_item32 * theta_company3[:, j][:, np.newaxis], k_vec)
        uv_deploy2[:, j] = np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ]
                                  * theta_item42 * theta_company4[:, j][:, np.newaxis], k_vec)
        uv_deploy3[:, j] = np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * 
                                  theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
    theta_deploy1 = theta_i[:, allocation1][item_ffc11, ] * theta_i[:, allocation2][item_ffc21, ]
    theta_deploy2 = theta_i[:, allocation1][item_fcf11, ] * theta_i[:, allocation2][item_fcf21, ]
    theta_deploy3 = theta_v[:, allocation1][company_cfc11, ] * theta_i[:, allocation2][item_cfc1, ]

    for j in range(k):
        #モデル誤差を定義
        y1 = U[2] - beta[2] - np.dot(uv_deploy1[:, allocation_omega[j, ]], np.repeat(1, k-1))
        y2 = U[3] - beta[3] - np.dot(uv_deploy2[:, allocation_omega[j, ]], np.repeat(1, k-1))
        y3 = U[4] - beta[4] - np.dot(uv_deploy3[:, allocation_omega[j, ]], np.repeat(1, k-1))

        #入力変数を定義
        x1 = theta_deploy1 * theta_company3[:, j][:, np.newaxis]
        x2 = theta_deploy2 * theta_company4[:, j][:, np.newaxis]
        x3 = theta_deploy3 * theta_company52[:, j][:, np.newaxis]

        #事後分布の期待値
        xy1 = np.dot(x1.T, y1); xy2 = np.dot(x2.T, y2); xy3 = np.dot(x3.T, y3)
        xxv1 = np.dot(x1.T, x1) + inv_Cov_m; xxv2 = np.dot(x2.T, x2) + inv_Cov_m; xxv3 = np.dot(x3.T, x3) + inv_Cov_m
        inv_xxv1 = np.linalg.inv(xxv1); inv_xxv2 = np.linalg.inv(xxv2); inv_xxv3 = np.linalg.inv(xxv3)
        theta_mu1 = np.dot(inv_xxv1, xy1); theta_mu2 = np.dot(inv_xxv2, xy2); theta_mu3 = np.dot(inv_xxv3, xy3)

        #多変量正規分布からパラメータをサンプリング
        omega_ffc[:, :, j] = np.random.multivariate_normal(theta_mu1, Sigma*inv_xxv1, 1).reshape(k, k)
        omega_fcf[:, :, j] = np.random.multivariate_normal(theta_mu2, Sigma*inv_xxv2, 1).reshape(k, k)
        omega_cfc[:, :, j] = np.random.multivariate_normal(theta_mu3, Sigma*inv_xxv3, 1).reshape(k, k)

        #期待値を更新
        uv_deploy1[:, j] = np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ]
                                  * theta_item32 * theta_company3[:, j][:, np.newaxis], k_vec)
        uv_deploy2[:, j] = np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ]
                                  * theta_item42 * theta_company4[:, j][:, np.newaxis], k_vec)
        uv_deploy3[:, j] = np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * 
                                  theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
        

    ##サンプリングしたパラメータを格納   
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep, ] = beta 
        THETA_V[:, :, mkeep] = theta_v
        THETA_I[:, :, mkeep] = theta_i
        OMEGA_FC[:, :, mkeep] = omega_fc 
        OMEGA_FF[:, :, mkeep] = omega_ff
        OMEGA_FFC[:, :, :, mkeep] = omega_ffc 
        OMEGA_FCF[:, :, :, mkeep] = omega_fcf
        OMEGA_CFC[:, :, :, mkeep] = omega_cfc
    
    ##対数尤度の更新とサンプリング結果の表示
    if rp%disp==0:
        ##学習データの対数尤度を更新
        #タッカー分解の期待値
        uv11 = np.dot(np.dot(theta_i, omega_fc)[item_fc1, ] * theta_company1, k_vec)
        uv12 = np.dot(np.dot(theta_i, omega_ff)[item_ff11] * (theta_item22), k_vec)
        uv13 = np.array(np.repeat(0.0, N31), dtype="float32")
        uv14 = np.array(np.repeat(0.0, N41), dtype="float32")
        uv15 = np.array(np.repeat(0.0, N51), dtype="float32")
        for j in range(k):
            uv13 += np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc11, ] * theta_item32 * theta_company3[:, j][:, np.newaxis], k_vec)
            uv14 += np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf11, ] * theta_item42 * theta_company4[:, j][:, np.newaxis], k_vec)
            uv15 += np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc11, ] * theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
        uv1 = [uv11, uv12, uv13, uv14, uv15]

        #対数尤度を更新
        mu1 = [j for j in range(type_n)]
        Prob1 = [j for j in range(type_n)]
        LL1 = np.repeat(0.0, type_n)
        for j in range(type_n):
            mu1[j] = beta[j] + uv1[j]
            Prob = norm.cdf(mu1[j], 0, Sigma)
            Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
            Prob1[j] = Prob
            LL1[j] = np.sum(y_learn[j]*np.log(Prob)) + np.sum((1-y_learn[j])*np.log(1-Prob))


        ##テストデータの対数尤度を更新
        #特徴ベクトルをidで対応付ける
        theta_item1 = theta_i[item_fc2, ]; theta_company1 = theta_v[company_fc2, ]
        theta_item21 = theta_i[item_ff12, ]; theta_item22 = theta_i[item_ff22, ]
        theta_item31 = theta_i[item_ffc12, ]; theta_item32 = theta_i[item_ffc22, ]; theta_company3 = theta_v[company_ffc2, ]
        theta_item41 = theta_i[item_fcf12, ]; theta_item42 = theta_i[item_fcf22, ]; theta_company4 = theta_v[company_fcf2, ]
        theta_item5 = theta_i[item_cfc2, ]; theta_company51 = theta_v[company_cfc12, ]; theta_company52 = theta_v[company_cfc22, ]

        #タッカー分解の期待値
        uv01 = np.dot(np.dot(theta_i, omega_fc)[item_fc2, ] * theta_company1, k_vec)
        uv02 = np.dot(np.dot(theta_i, omega_ff)[item_ff12] * (theta_item22), k_vec)
        uv03 = np.array(np.repeat(0.0, N32), dtype="float32")
        uv04 = np.array(np.repeat(0.0, N42), dtype="float32")
        uv05 = np.array(np.repeat(0.0, N52), dtype="float32")
        for j in range(k):
            uv03 += np.dot(np.dot(theta_i, omega_ffc[:, :, j])[item_ffc12, ] * theta_item32 * theta_company3[:, j][:, np.newaxis], k_vec)
            uv04 += np.dot(np.dot(theta_i, omega_fcf[:, :, j])[item_fcf12, ] * theta_item42 * theta_company4[:, j][:, np.newaxis], k_vec)
            uv05 += np.dot(np.dot(theta_v, omega_cfc[:, :, j])[company_cfc12, ] * theta_item5 * theta_company52[:, j][:, np.newaxis], k_vec)
        uv0 = [uv01, uv02, uv03, uv04, uv05]

        #対数尤度を更新
        mu0 = [j for j in range(type_n)]
        Prob0 = [j for j in range(type_n)]
        LL0 = np.repeat(0.0, type_n)
        for j in range(type_n):
            mu0[j] = beta[j] + uv0[j]
            Prob = norm.cdf(mu0[j], 0, Sigma)
            Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
            Prob0[j] = Prob
            LL0[j] = np.sum(y_test[j]*np.log(Prob)) + np.sum((1-y_test[j])*np.log(1-Prob))


        #サンプリング結果を表示
        print(rp)
        print(np.round(np.append(LL1, np.sum(LL1)), 1))
        print(np.round(np.append(LL0, np.sum(LL0)), 1))

0
[ -14029.6  -15756.1  -58047.5  -75281.3 -113898.7 -277013.1]
[ -3528.5  -3949.6 -14592.3 -19155.2 -28585.2 -69810.8]
10
[ -5694.1 -10943.5 -16091.4 -16635.3 -19010.6 -68374.9]
[ -1660.7  -3124.2  -4696.4  -4845.6  -5226.3 -19553.2]
20
[ -4341.8  -9269.3 -10727.  -10685.5 -11989.6 -47013.1]
[ -1421.8  -2965.7  -3461.5  -3348.   -3615.4 -14812.3]
30
[ -3940.4  -8617.5  -8643.7  -8721.   -9755.3 -39677.9]
[ -1389.   -2886.5  -3168.8  -2980.6  -3120.8 -13545.7]
40
[ -3588.2  -8432.1  -7654.   -7496.4  -8547.5 -35718.2]
[ -1313.5  -2822.9  -3045.8  -2685.6  -2945.1 -12813. ]
50
[ -3495.7  -8015.9  -6966.1  -6882.4  -7713.9 -33074. ]
[ -1260.8  -2803.9  -2870.4  -2560.2  -2798.7 -12294. ]
60
[ -3280.2  -8014.9  -6570.   -6178.7  -7164.5 -31208.2]
[ -1239.4  -2860.2  -2851.7  -2443.6  -2637.  -12031.9]
70
[ -3145.5  -7832.8  -6284.2  -5826.4  -6735.3 -29824.2]
[ -1301.8  -2837.6  -2825.2  -2366.6  -2647.8 -11979.1]
80
[ -3036.   -7748.6  -5876.2  -5617.3  -6393.5 -28671.7]
[ -1213.3  -2892

LinAlgError: SVD did not converge

In [46]:
#事後平均を計算
RS = np.arange(burnin, int(500/keep))
theta_i = np.mean(THETA_I[:, :, RS], axis=2)
theta_v = np.mean(THETA_V[:, :, RS], axis=2)

In [71]:
#アイテム間の類似度
q = np.sqrt(np.sum(np.power(theta_i, 2), axis=1))[:, np.newaxis]
similar = np.dot(theta_i, theta_i.T) / np.dot(q, q.T)

i = 13
sortlist = np.argsort(similar[i, ])[::-1]
similar_df = item_df.iloc[sortlist]; similar_df.index = np.arange(similar_df.shape[0])
similar_df["score"] = similar[i, ][sortlist]
similar_df

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,item,score
0,13,8Kカメラ,1.000000
1,1583,内臓カメラ,0.931540
2,758,スマートフォンケース,0.930418
3,476,カメラ,0.906767
4,481,カラープリンター,0.882479
5,2879,配向膜,0.875719
6,148,LEDディスプレイ,0.875529
7,137,ITコンサルティングサービス,0.874593
8,1589,内視鏡,0.866620
9,438,オフィス用インクジェットプリンタ,0.860621


In [72]:
#企業間の類似度
q = np.sqrt(np.sum(np.power(theta_v, 2), axis=1))[:, np.newaxis]
similar = np.dot(theta_v, theta_v.T) / np.dot(q, q.T)

i = 13
sortlist = np.argsort(similar[i, ])[::-1]
similar_df = company_df.iloc[sortlist]; similar_df.index = np.arange(similar_df.shape[0])
similar_df["score"] = similar[i, ][sortlist]
similar_df

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,ticker,company,score
0,13,1518,三井松島産業,1.000000
1,1598,9675,常磐興産,0.924119
2,656,5721,エス・サイエンス,0.860645
3,641,5541,大平洋金属,0.857434
4,1359,8075,神鋼商事,0.840266
5,402,4102,丸尾カルシウム,0.839590
6,631,5446,北越メタル,0.837104
7,480,4629,大伸化学,0.811304
8,825,6331,三菱化工機,0.809554
9,1345,8031,三井物産,0.808173


id            0
ticker     1332
company    日本水産
Name: 0, dtype: object

In [80]:
#企業とアイテム間の類似度
q1 = np.sqrt(np.sum(np.power(theta_v, 2), axis=1))[:, np.newaxis]
q2 = np.sqrt(np.sum(np.power(theta_i, 2), axis=1))[:, np.newaxis]
similar = np.dot(theta_v, theta_i.T) / np.dot(q1, q2.T)

i = 1
print(company_df.iloc[i])
sortlist = np.argsort(similar[i, ])[::-1]
similar_df = item_df.iloc[sortlist]; similar_df.index = np.arange(similar_df.shape[0])
similar_df["score"] = similar[i, ][sortlist]
similar_df

id             1
ticker      1376
company    カネコ種苗
Name: 1, dtype: object


C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,id,item,score
0,935,ドライバーIC,0.718732
1,718,スクラブ洗浄装置,0.708969
2,646,サーマルチャンバ,0.704001
3,286,アナログIC,0.683331
4,2506,腕時計用部品,0.679940
5,965,ハイブリッドボンディング装置,0.678347
6,986,バイオ医薬品中間体,0.671694
7,40,CD-SEM,0.662090
8,2443,窒化ガリウム,0.660459
9,1714,反射率測定機,0.648797


In [42]:
RS

array([100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 151., 152., 153., 154.,
       155., 156., 157., 158., 159., 160., 161., 162., 163., 164., 165.,
       166., 167., 168., 169., 170., 171., 172., 173., 174., 175., 176.,
       177., 178., 179., 180., 181., 182., 183., 184., 185., 186., 187.,
       188., 189., 190., 191., 192., 193., 194., 195., 196., 197., 198.,
       199., 200., 201., 202., 203., 204., 205., 206., 207., 208., 209.,
       210., 211., 212., 213., 214., 215., 216., 217., 218., 219., 220.,
       221., 222., 223., 224., 225., 226., 227., 228., 229., 230., 231.,
       232., 233., 234., 235., 236., 237., 238., 239., 240., 241., 242.,
       243., 244., 245., 246., 247., 248., 249.])

In [36]:
np.random.multivariate_normal(theta_mu2, Sigma*inv_xxv2, 1)

LinAlgError: SVD did not converge

In [39]:
np.dot(inv_xxv2, xy2)

array([-1.15858808,  1.53949547, -0.49633062, -0.67286203, -0.16117576,
       -0.26075954, -0.42075831,  0.07174537,  0.03449246,  0.40525198,
        1.47956921, -0.34824412,  0.74082313,  1.49173005,  0.19398219,
        0.11089045,  0.15760142, -0.31473991,  0.03936092,  0.33916489,
       -0.44508464,  0.67367826, -0.18465807, -0.27683114,  0.068187  ,
       -0.13590956, -0.24359427,  0.17995336,  0.22274635, -0.11082103,
       -0.41455117,  1.50056073, -0.42543639,  0.42970428,  0.69004264,
        0.04669724,  0.30952761, -0.03421802,  0.49346861,  0.85510587,
        0.10753888,  0.18273276,  0.32465272,  0.88814876,  0.99560161,
       -0.20019097, -0.15656392, -0.08470082,  0.18450486,  0.50232503,
       -0.09301809,  0.31194768, -0.17459097, -0.10903603, -0.08143711,
        0.25204471, -0.08774241,  0.02103289, -0.17402278,  0.20387805,
       -0.37748194,  0.36064632, -0.24692435,  0.14092049,  0.03430019,
        0.0619551 ,  0.09309346,  0.13295135,  0.24797113,  0.15